In [19]:
import os
from pathlib import Path
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    r2_score,
    mean_absolute_percentage_error
)

In [20]:
csv_file_path = '/content/BTC_USD.csv'

# Extract base name without extension for naming
csv_base = Path(csv_file_path).stem

# Define paths for saving the best model and its metrics
model_history_dir = Path('model_history')
result_history_dir = Path('result_history')

os.makedirs(model_history_dir, exist_ok=True)
os.makedirs(result_history_dir, exist_ok=True)

best_model_filename = f"{csv_base}_best_model.pt"
best_model_path = model_history_dir / best_model_filename
metrics_path = result_history_dir / f"{csv_base}_metrics.txt"

forecast_history = 24  # Number of past time steps to use for input sequences.
forecast_horizon = 1  # Number of future time steps to predict (e.g., 0 means predicting the next step immediately).

batch_size = 64  # Number of samples in each mini-batch
max_iterations = 100
model_name = 'Arima'
order=(24,1,0)
best_mse = float('inf')
include_baseline = True

In [21]:
def log_print(*args, **kwargs):
    with open(metrics_path, 'a') as metrics_file:
        msg = ' '.join(map(str, args))
        print(msg)
        metrics_file.write(msg + '\n')

Pereprocess

In [22]:
# Load Data
try:
  df_raw = pd.read_csv(csv_file_path)
  print("CSV file loaded successfully.")
except Exception as e:
  print(f"Error reading CSV file: {e}")

df_raw.head(10)

CSV file loaded successfully.


,Date,Price,Open,High,Low,Vol.,Change %
0,01/26/2025,105000.0,104700.0,105440.0,104510.0,0.11K,0.29%
1,01/25/2025,104700.0,104800.0,105260.0,104100.0,0.14K,-0.10%
2,01/24/2025,104800.0,103930.0,107030.0,102720.0,0.52K,0.84%
3,01/23/2025,103930.0,103680.0,106780.0,101260.0,1.70K,0.24%
4,01/22/2025,103680.0,106150.0,106290.0,103400.0,0.51K,-2.33%
5,01/21/2025,106150.0,102170.0,107360.0,100140.0,1.12K,3.90%
6,01/20/2025,102170.0,101360.0,109410.0,99634.0,2.92K,0.80%
7,01/19/2025,101360.0,104490.0,106370.0,99675.0,1.04K,-3.00%
8,01/18/2025,104490.0,104040.0,104950.0,102200.0,0.54K,0.43%
9,01/17/2025,104040.0,100040.0,105800.0,100000.0,1.26K,4.00%


In [23]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3677 entries, 0 to 3676
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      3677 non-null   object 
 1   Price     3677 non-null   float64
 2   Open      3677 non-null   float64
 3   High      3677 non-null   float64
 4   Low       3677 non-null   float64
 5   Vol.      3667 non-null   object 
 6   Change %  3677 non-null   object 
dtypes: float64(4), object(3)
memory usage: 201.2+ KB


In [24]:
# Preprocess data
datetime_col = "Date"
freq = '1D'
columns = ['Price']
method = 'ffill'
value = 0


# Define a data preprocessing pipeline
preproc_pipe = sklearn.pipeline.Pipeline([
    # Step 1: Drop duplicate rows based on the datetime column
    ('drop_duplicates', sklearn.preprocessing.FunctionTransformer(lambda df: df.drop_duplicates(subset=datetime_col))),

    # Step 2: Fill missing values in the dataset
    ('fill_missing', sklearn.preprocessing.FunctionTransformer(
        lambda df: df.fillna(method=method).fillna(value=value)  # Fill missing values using a specified method and value
    ))
])

# Apply the pipeline to the raw data
df = preproc_pipe.fit_transform(df_raw)

# Convert the datetime column to a standard datetime format
df[datetime_col] = pd.to_datetime(df[datetime_col])

# Sort the data by the datetime column in ascending order
df = df.sort_values(by=datetime_col)

<ipython-input-24-5226ef2863c8>:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda df: df.fillna(method=method).fillna(value=value)  # Fill missing values using a specified method and value


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3677 entries, 3676 to 0
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      3677 non-null   datetime64[ns]
 1   Price     3677 non-null   float64       
 2   Open      3677 non-null   float64       
 3   High      3677 non-null   float64       
 4   Low       3677 non-null   float64       
 5   Vol.      3677 non-null   object        
 6   Change %  3677 non-null   object        
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 229.8+ KB


In [26]:
# Define the proportions for test and validation sets
test_size = 0.05  # Fraction of the data to be used as the test set
valid_size = 0.05  # Fraction of the data to be used as the validation set

# Calculate the total number of samples in the dataset
total_size = len(df)  # The total number of rows in the DataFrame

# Calculate the number of samples for the test set
test_split = int(total_size * test_size)  # Convert the test size fraction to the actual count

# Calculate the number of samples for the validation set
valid_split = int(total_size * valid_size)  # Convert the validation size fraction to the actual count

# Calculate the end index of the training set (exclusive)
train_end = total_size - test_split - valid_split  # Determine where the training set ends

# Calculate the end index of the validation set (exclusive)
valid_end = total_size - test_split  # Determine where the validation set ends

# Split the data into training, validation, and test sets using indices
train_df = df.iloc[:train_end]  # Select rows for the training set
valid_df = df.iloc[train_end:valid_end]  # Select rows for the validation set
test_df = df.iloc[valid_end:]  # Select rows for the test set

# Print the sizes of each split for verification
print(f"Data split into train ({len(train_df)}), validation ({len(valid_df)}), and test ({len(test_df)}) sets.")


Data split into train (3311), validation (183), and test (183) sets.


In [27]:
# Define the columns to be scaled
scale_columns = ['Price']  # Specify the column(s) to scale, in this case 'Price'

# Initialize the MinMaxScaler with a feature range of 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit the scaler using the training data
scaler.fit(train_df[scale_columns])  # Calculate the min and max values from the training set for scaling

# Transform the training data using the fitted scaler
train_scaled = scaler.transform(train_df[scale_columns])  # Scale the 'Price' column in the training set

# Transform the validation data using the same scaler
valid_scaled = scaler.transform(valid_df[scale_columns])  # Apply the same scaling to the validation set

# Transform the test data using the same scaler
test_scaled = scaler.transform(test_df[scale_columns])  # Apply the same scaling to the test set

# Print a message indicating that the scaling is complete
print("Data scaling completed.")


Data scaling completed.


In [28]:
train_scaled.shape
order

(24, 1, 0)

###Train

In [29]:
log_print(f"Model Name: {model_name}")
log_print(f"Sequence Length (Forecast History): {forecast_history}")
log_print(f"Forecast Horizon: {forecast_horizon}")
log_print(f"Training Started at: {pd.Timestamp.now()}")
for i in range(max_iterations):
            log_print(f"\nStarting training iteration {i + 1}/{max_iterations}")

            try:
                model = SARIMAX(train_scaled,
                                order=order,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
                model_fit = model.fit(disp=False)
                log_print(f"ARIMA model fitted: order={order}")
            except Exception as e:
                log_print(f"Error fitting ARIMA model: {e}")
                continue


            # Validate the model
            try:
                # Forecast validation period
                preds_valid_scaled = model_fit.predict(start=train_end, end=valid_end -1)
                preds_valid_scaled = preds_valid_scaled.reshape(-1, 1)
                preds_valid = scaler.inverse_transform(preds_valid_scaled)
                actual_valid = scaler.inverse_transform(valid_scaled)

                # Calculate validation MSE
                val_mse = mean_squared_error(actual_valid, preds_valid)
                log_print(f"Iteration {i + 1}: Validation MSE: {val_mse:.6f}")

                # Check if this is the best model so far
                if val_mse < best_mse:
                    best_mse = val_mse
                    # Save the model
                    with open(best_model_path, 'wb') as f:
                        pickle.dump(model_fit, f)
                    log_print(f"New best model saved at '{best_model_path}' with Validation MSE {best_mse:.6f}")
            except Exception as e:
                log_print(f"Error during validation prediction: {e}")

 # After training iterations, log the best model info
log_print(f"\nTraining completed. Best Validation MSE: {best_mse:.6f}")
log_print(f"Best model saved at: '{best_model_path}'")



log_print(f"Metrics saved at '{metrics_path}'")

Model Name: Arima
Sequence Length (Forecast History): 24
Forecast Horizon: 1
Training Started at: 2025-05-28 05:45:55.964762

Starting training iteration 1/100
ARIMA model fitted: order=(24, 1, 0)
Iteration 1: Validation MSE: 489789105.517301
New best model saved at 'model_history/BTC_USD_best_model.pt' with Validation MSE 489789105.517301

Starting training iteration 2/100
ARIMA model fitted: order=(24, 1, 0)
Iteration 2: Validation MSE: 489789105.517301

Starting training iteration 3/100
ARIMA model fitted: order=(24, 1, 0)
Iteration 3: Validation MSE: 489789105.517301

Starting training iteration 4/100
ARIMA model fitted: order=(24, 1, 0)
Iteration 4: Validation MSE: 489789105.517301

Starting training iteration 5/100
ARIMA model fitted: order=(24, 1, 0)
Iteration 5: Validation MSE: 489789105.517301

Starting training iteration 6/100
ARIMA model fitted: order=(24, 1, 0)
Iteration 6: Validation MSE: 489789105.517301

Starting training iteration 7/100
ARIMA model fitted: order=(24, 1,

###Evalute

In [30]:
with open(best_model_path, 'rb') as f:
    model_fit = pickle.load(f)

In [31]:
# Forecast test period
preds_test_scaled = model_fit.predict(start=train_end, end=train_end + len(test_df) -1)
preds_test_scaled = preds_test_scaled.reshape(-1, 1)
preds_test = scaler.inverse_transform(preds_test_scaled)
actual_test = scaler.inverse_transform(test_scaled)

# Calculate metrics
mse_val = mean_squared_error(actual_test, preds_test)
mae_val = mean_absolute_error(actual_test, preds_test)
rmse_val = np.sqrt(mse_val)
r2_val = r2_score(actual_test, preds_test)
mape_val = mean_absolute_percentage_error(actual_test, preds_test)

log_print("\nTest Regression Metrics:")
log_print(f"\nFinal Test MSE: {mse_val:.6f}")
log_print(f"Final Test MAE: {mae_val:.6f}")
log_print(f"  R2   : {r2_val:.6f}")
log_print(f"  MAPE : {mape_val:.6f}")

# Baseline Predictions: Last value from training set
last_known = scaler.inverse_transform(train_scaled[-1].reshape(1, -1))
baseline_predictions = np.array([last_known.flatten()[0]] * len(test_df)).reshape(-1, 1)

test_mse_baseline = mean_squared_error(actual_test, baseline_predictions)
test_mae_baseline = mean_absolute_error(actual_test, baseline_predictions)
test_rmse_baseline = np.sqrt(test_mse_baseline)

# Log Test Metrics
log_print("\nTest Metrics:")
log_print("-------------")
if include_baseline:
    log_print("Baseline Model:")
    log_print(f"  MAE  : {test_mae_baseline:.6f}")
    log_print(f"  MSE  : {test_mse_baseline:.6f}")
    log_print(f"  RMSE : {test_rmse_baseline:.6f}\n")
log_print(f"{model_name} Model:")
log_print(f"  MAE  : {mae_val:.6f}")
log_print(f"  MSE  : {mse_val:.6f}")
log_print(f"  RMSE : {rmse_val:.6f}\n")

if include_baseline:
    # Compare model and baseline in terms of MSE
    mse_diff = mse_val - test_mse_baseline
    mse_percentage = (mse_diff / test_mse_baseline) * 100

    if mse_percentage > 0:
        log_print(f"Model MSE is worse by {mse_percentage:.2f}% compared to the baseline.")
    else:
        log_print(f"Model MSE is better by {abs(mse_percentage):.2f}% compared to the baseline.")




Test Regression Metrics:

Final Test MSE: 1603100671.910398
Final Test MAE: 35974.790722
  R2   : -4.191538
  MAPE : 0.435601

Test Metrics:
-------------
Baseline Model:
  MAE  : 35835.633880
  MSE  : 1592983719.721312
  RMSE : 39912.200136

Arima Model:
  MAE  : 35974.790722
  MSE  : 1603100671.910398
  RMSE : 40038.739639

Model MSE is worse by 0.64% compared to the baseline.


In [32]:
from google.colab import drive
drive.mount('/content/drive')
try:
  results = pd.read_csv('/content/drive/MyDrive/bitcoin/results.csv')
except Exception as e:
  print(e)
  results = pd.DataFrame(columns=['model','MSE', 'MAE','R2_score','MAPE','time frame', 'type'])

results.loc[len(results)] = [model_name, mse_val, mae_val, r2_val, mape_val, '2015_2025' , 'h']
results.to_csv('/content/drive/MyDrive/bitcoin/results.csv',index=False)
results

ValueError: mount failed

###Figure

In [ ]:
indices = np.arange(len(actual_test))

# Create the plot
plt.figure(figsize=(14, 7))

# Plot Actual Prices
plt.plot(indices, actual_test, label='Actual Price', marker='o', linestyle='-', color='blue')

# Plot Predicted Prices from ARIMA Model
plt.plot(indices, preds_test, label=f'{model_name} Predictions', marker='x', linestyle='--', color='red')

if include_baseline:
    # Plot Baseline Predictions
    plt.plot(indices, baseline_predictions, label='Baseline Prediction', marker='s', linestyle=':', color='green')

# Enhancements
plt.xlabel('Sample Index', fontsize=14)
plt.ylabel('Price', fontsize=14)
plt.title(f'Actual vs. Predicted{" vs. Baseline" if include_baseline else ""} Prices (Test Set)',
          fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()
# Save the plot with forecast_history in the filename
plot_filename = f"{csv_base}_{forecast_history}_{forecast_horizon}_test_plot.png"
plot_path = result_history_dir / plot_filename
plt.savefig(plot_path)
plt.close()
log_print(f"Plot saved to '{plot_path}'")




log_print(f"\nEvaluation completed for '{csv_base}' with model '{model_name}' and forecast_history={forecast_history}.")
log_print(f"Metrics and plot appended to '{metrics_path}' and saved plot at '{plot_path}'")